In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
class RemoveCorrelationTransformer2(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7):
        self.correlation_threshold = correlation_threshold


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson', min_periods=1)
        df_not_correlated = ~(df_corr.mask(
            np.tril(np.ones([len(df_corr)] * 2, dtype=bool))).abs() > self.correlation_threshold).any()
        self.un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        df = df[self.un_corr_idx]
        return df.values

In [3]:

class RemoveMissingFeaturesTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, Y=None):
        self.is_missing = X.isnull().values.any(axis=0)
        return self

    def transform(self, X, Y=None):
        copy_x = pd.DataFrame(X)
        self.is_missing += copy_x.isnull().values.any(axis=0)

        copy_x = copy_x.iloc[:, ~self.is_missing]

        return copy_x.values


In [4]:
def refactor_labels(df):
    return df.replace({'low': 0 ,'high': 1, 'clinical': 1 })


def get_data(file_name, LSAS_threshold=None):
    group_column = 'group'
    sub_num_col = 'Subject_Number'
    lsas_col = 'LSAS'
    df = pd.read_excel(file_name, sheet_name='Sheet1')
    if LSAS_threshold is None:
        X = df.drop([group_column, sub_num_col, lsas_col], 1)
        Y = refactor_labels(df[group_column])
        return X, Y
    else:
        X = df.drop([group_column], 1)
        Y = pd.Series(np.where(X[lsas_col] > LSAS_threshold, 1, 0))
        X = X.drop([sub_num_col, lsas_col], 1)
        return X, Y


## get training data

In [5]:
file_name = "training_set_100.xlsx"

In [6]:
X_train, y_train = get_data(file_name, LSAS_threshold = 50)

In [7]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, stratify=y_train)


## get test data

In [8]:
file_name = "full_test_set.xlsx"
df = pd.read_excel(file_name, sheet_name='Sheet1')
X_test = df.drop('Subject_Number', axis = 1)

## training pipeline

In [49]:
pipe =  Pipeline([
    ("rnf", RemoveMissingFeaturesTransformer()), 
   # ('scaling', MinMaxScaler()),
   # ('correlation_threshold', RemoveCorrelationTransformer2()), 
  #  ('pca', PCA()),
     ('rfc', RFE(RandomForestClassifier(n_estimators = 100))),
                ('classifier', XGBClassifier())])

In [83]:
params_grid = [
    {
       # 'correlation_threshold__correlation_threshold' : [0.85], 
       #'pca__n_components': [0.9],
        'classifier__min_child_weight': [1],
        'classifier__gamma': [2, 1.5, 1.85, 1.7],
        'classifier__subsample': [0.9],
        'classifier__colsample_bytree': [0.7],
        'classifier__max_depth': [3],
        'classifier__reg_alpha' : [0.3],
        'classifier__reg_lambda' : [0.7],
        #'classifier__scale_pos_weight' : [0, 1],
    'classifier__learning_rate': [0.01],
        'rfc__n_features_to_select': [16], #so called `eta` value
              'classifier__n_estimators': [100]}] #number of trees, change it to 1000 for better results
  #  'rfc__n_features_to_select': [8, 12]}]

In [84]:
loo = LeaveOneOut()
gs = GridSearchCV(pipe, params_grid, cv=loo, scoring='accuracy')
gs.fit(X_train_2, y_train_2)

GridSearchCV(cv=LeaveOneOut(), error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('rnf', RemoveMissingFeaturesTransformer()), ('correlation_threshold', RemoveCorrelationTransformer2(correlation_threshold=0.7)), ('rfc', RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=N...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'classifier__min_child_weight': [1], 'classifier__gamma': [2, 1.5, 1.85, 1.7], 'classifier__subsample': [0.9], 'classifier__colsample_bytree': [0.7], 'classifier__max_depth': [3], 'classifier__reg_alpha': [0.3], 'classifier__reg_lambda': [0.7], 'classifier__learning_rate': [0.01], 'rfc__n_features_to_select': [16], 'classifier__n_estimators': [100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=

In [85]:
gs.best_params_

{'classifier__colsample_bytree': 0.7,
 'classifier__gamma': 2,
 'classifier__learning_rate': 0.01,
 'classifier__max_depth': 3,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 100,
 'classifier__reg_alpha': 0.3,
 'classifier__reg_lambda': 0.7,
 'classifier__subsample': 0.9,
 'rfc__n_features_to_select': 16}

In [86]:
gs.best_score_

0.5934065934065934

In [82]:
results = []
k = 10
for i in range(k):
    loo = LeaveOneOut()
    gs = GridSearchCV(pipe, params_grid, cv=loo, scoring='accuracy')
    gs.fit(X_train, y_train)
    results.append(gs.best_score_)
sum(results)/k

KeyboardInterrupt: 

## holdout

In [ ]:
model = gs.fit(X_train_2, y_train_2)

In [ ]:
y_pred = model.predict(X_test_2)

In [ ]:
accuracy_score(y_pred, y_test_2)

## train the model

In [45]:
results = []
for i in range(20):
    loo = LeaveOneOut()
    gs = GridSearchCV(pipe, params_grid, cv=loo, scoring='accuracy')
    gs.fit(X_train, y_train)
    results.append(gs.best_score_)

In [46]:
gs.best_params_

{'classifier__colsample_bytree': 0.7,
 'classifier__gamma': 1.5,
 'classifier__learning_rate': 0.05,
 'classifier__max_depth': 3,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 500,
 'classifier__reg_alpha': 0.5,
 'classifier__reg_lambda': 0.5,
 'classifier__subsample': 0.9,
 'correlation_threshold__correlation_threshold': 0.9,
 'rfc__n_features_to_select': 18}

In [47]:
gs.best_score_

0.7352941176470589

## create predctions

In [48]:
sum(results)/20

0.7352941176470585

In [20]:
X_train.shape

(102, 25)